In [1]:
#@title　步驟一：輸入授權密碼

!git clone https://github.com/chenkenanalytic/gpt2_jingyong.git
from IPython.display import clear_output
clear_output(wait=True)
!7z x /content/gpt2_jingyong/aifreeteam_GPT2.7z.001
clear_output(wait=True)

import os
try:
  if 'cache' in os.listdir('/content/aifreeteam_GPT2'):
    print('授權成功！')
  else:
    print('授權失敗，授權問題請洽 AI . FREE Team (ai.free.team@gmail.com)')
except:
  print('授權失敗，授權問題請洽 AI . FREE Team (ai.free.team@gmail.com)')

授權成功！


In [2]:
#@title　步驟二：安裝 transformers 及相關套件
from IPython.display import clear_output
!pip install transformers
!pip install tqdm
!pip install torchsnooper
clear_output()

#基本 import
import sys
import json
import torch
import textwrap
import torchsnooper
import transformers
import torch.nn.functional as F
from tqdm import trange
from IPython.core.display import display, HTML

#下載中文 GPT-Chinese 套件
GITHUB_REPO = "GPT2-Chinese"
!rm -rf {GITHUB_REPO}
!git clone https://github.com/Morizeyao/{GITHUB_REPO}.git {GITHUB_REPO}
if not GITHUB_REPO in sys.path:
    sys.path += [GITHUB_REPO]
clear_output()

In [3]:
#@title 步驟三：設定模型資訊

config_file = '/content/aifreeteam_GPT2/model/10layers_12heads_1024len_768embd_full_corpus_16bsize/config.json'
model_ckpt = "/content/aifreeteam_GPT2/model/10layers_12heads_1024len_768embd_full_corpus_16bsize/pytorch_model.bin"
vocab_file = "/content/aifreeteam_GPT2/cache/vocab_small.txt"

from tokenizations import tokenization_bert

# make model output attentions
config = transformers.GPT2Config.from_json_file(config_file)
config.output_attentions = True

model = transformers.GPT2LMHeadModel.from_pretrained(model_ckpt, config=config)
tokenizer = tokenization_bert.BertTokenizer(vocab_file=vocab_file)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()
clear_output()

In [4]:
#@title 步驟四：生成文字前置工具開發

def is_word(word):
    for item in list(word):
        if item not in 'qwertyuiopasdfghjklzxcvbnm':
            return False
    return True

def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def fast_sample_sequence(model,context,length,temperature=1, top_k=0, top_p=0.0,device='cpu'):
    inputs = torch.LongTensor(context).view(1, -1).to(device)
    if len(context) > 1:
        _, past = model(inputs[:, :-1], None)[:2]
        prev = inputs[:, -1].view(1, -1)
    else:
        past = None
        prev = inputs
    generate = [] + context
    with torch.no_grad():
        for i in trange(length):
            output = model(prev, past=past)
            output, past = output[:2]
            output = output[-1].squeeze(0) / temperature
            filtered_logits = top_k_top_p_filtering(output, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(torch.softmax(filtered_logits, dim=-1), num_samples=1)
            
            # redraw if [UNK]
            if next_token.unsqueeze(0) != 100:
                generate.append(next_token.item())
                prev = next_token.view(1, 1)

    return generate


def get_html(context, generated_text, algorithm=''):
    if generated_text[-1] != '。':
        if generated_text[-1] == '，':
            generated_text= generated_text[:-1]
        generated_text += ' ...'
    
    html = """
    <html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <style>
    * {
      box-sizing: border-box;
    }

    /* Create two unequal columns that floats next to each other */
    .column {
      float: left;
      padding: 10px;
    }

    .left {
      width: 25%;
    }

    .right {
      width: 70%;
    }

    /* Clear floats after the columns */
    .row:after {
      content: "";
      display: table;
      clear: both;
    }
    </style>
    </head>
    <body>

    <div style="background-color:#FFF !important">
        <div class="row">
          <div class="column left">
            <h2 style="color: #3FB0F0;text-align:center;font-family:Microsoft JhengHei">
                前文段落創作
            </h2>
          </div>
          <div class="column right">
            <h3 style="color: black;line-height: 1.5;font-family:Microsoft JhengHei">context</h3>
          </div>
        </div>

        <hr/>

        <div class="row">
          <div class="column left">
            <h2 style="color: #8DCD49;text-align:center;font-family:Microsoft JhengHei">
                algorithm
                生成結果
            </h2>
          </div>
          <div class="column right">
            <h3 style="color: black;line-height: 1.5;font-family:Microsoft JhengHei">generated_text</h3>

          </div>
        </div>
    </div>

    </body>
    </html>


    """.replace('context', context).replace('generated_text', generated_text)
    
    if not algorithm:
        html = html.replace("algorithm", "")
    else:
        html = html.replace("algorithm", f'{algorithm}<br/>')
    
    
    return html


def generate(context, topk, topp, temperature, device, line_len=40):
    context_tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(context))
        
    # auto-regressive
    out = fast_sample_sequence(
        model=model, length=length,
        context=context_tokens,
        temperature=temperature, top_k=topk, top_p=topp, device=device
    )

    # rendering
    tokens = tokenizer.convert_ids_to_tokens(out)

    for i, item in enumerate(tokens):
        if item == '[MASK]':
            tokens[i] = ''
        if item == '[CLS]' or item == '[SEP]':
            tokens[i] = '\n'
    
    generated_text = ''.join(tokens).strip().replace(context, '')
    html = get_html(context, generated_text)
    
    return html, generated_text

In [5]:
# 生成參數設定

## 固定
nsamples = 1
batch_size = 1

## 固定可嘗試微調
"""
原參數：
length = model.config.n_ctx // 2
topk = 30
topp = 0
temperature = 1
"""

length = model.config.n_ctx // 2
topk = 30
topp = 0
temperature = 1

In [6]:
#前文段落創作

context = "這一番顛動，陸無雙早痛得死去活來，於楊過怎樣相救、怎樣抱著她藏身在牛腹之下、怎樣躍入草叢，她都是迷糊不清，過了好一陣，神智稍復，「啊」的一聲叫了出來。楊過忙按住她口，在她耳邊低聲道：「別作聲！」只聽腳步聲響，洪淩波道：「咦，怎地一霎眼就不見了人？」遠處李莫愁道：「咱們走吧。這小賤人定是逃得遠了。」但聽洪淩波的腳步聲漸漸遠去。陸無雙極是氣悶，又待呼痛，楊過仍是按住她嘴不放。"

In [7]:
#生成文章
html, generated_text = generate(context, topk, topp, temperature, device)
display(HTML(html))

100%|██████████| 512/512 [00:08<00:00, 59.94it/s]
